In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

In [6]:
train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./test.csv")

train_data.head()

,x,y
0,34,36.951713
1,77,75.385850
2,70,72.652601
3,55,53.178152
4,14,20.836313


In [3]:
print(f"Train Shape: {train_data.shape}")
print(f"Test Shape: {test_data.shape}")

Train Shape: (1000000, 2)
Test Shape: (1000000, 2)


In [7]:
X_train, y_train = train_data['x'][:10000], train_data['y'][:10000]
X_test, y_test = test_data['x'][:2000], test_data['y'][:2000]

print(f"Train data shape: {X_train.shape} | {y_train.shape}")
print(f"Test data shape: {X_test.shape} | {y_test.shape}")

Train data shape: (10000,) | (10000,)
Test data shape: (2000,) | (2000,)
